# Glacier edu module WIP
This is an introduction/walk through of the glacier module in the OGGM-Edu library.
At the moment it contains five classes: `GlacierBed`, `MassBalance`, `Glacier`, `SurgingGlacier` and `GlacierCollection`. 

#### Interactive boxes PSA

- Green
<div class="alert alert-success">
    <details>
        <summary>Just an example (Click me)</summary>
        This is where the bread of the box go.
    </details>
</div>
- Orange
<div class="alert alert-warning">
    <details>
        <summary>Just an example (Click me)</summary>
        This is where the bread of the box go.
    </details>
</div>
- Red
<div class="alert alert-danger">
    <details>
        <summary>Just an example (Click me)</summary>
        This is where the bread of the box go.
    </details>
</div>
- Blue
<div class="alert alert-info">
    <details>
        <summary>Just an example (Click me)</summary>
        This is where the bread of the box go.
    </details>
</div>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from oggm_edu import Glacier, GlacierCollection, GlacierBed, SurgingGlacier, MassBalance

## Glacier bed
The `GlacierBed` provides with a separate object for the bed of the glacier.
This is then passed to the initialisation of the glacier.
There are two ways to initialise a glacier bed:
- Pass single scalars to the arguments top, bottom and width.
    This creates a square bed.
- Pass multiple values (list/tuple) to altitudes and widths.
    These have to be the same length.
    Each pair corresponds to the width at that altitude.
    Gives more control over the geometry.

In [ ]:
# Create a complex bed.
bed = GlacierBed(altitudes=(3800, 3300, 3000, 100),
                 widths=(1200, 900, 800, 600), slope=0.2)
# Create a simple bed
# bed = GlacierBed(top=3400, bottom=1400, width=300)

In [ ]:
# Representation
bed

In [ ]:
# Plotting method.
bed.plot()

## Mass balance
This class is used to create the mass balance of the our glacier. 
For now it is simply a wrapper around the `LinearMassBalance` class of OGGM, with some extra attributes and methods for edu purposes.

In [ ]:
# We initialise it with an ELA and mass balance gradient.
mb = MassBalance(3000, 4)

In [ ]:
mb

## Glacier
The `Glacier` class gives us an object with methods and attributes which enables us to play around and diagnose a glacier.
We begin with the definition of our glacier.
It expects a bed of the type `GlacierBed` and a mass balance of the type `MassBalance` (this can be omitted).

In [ ]:
# Initialise the glacier with the bed and mass balance from earlier.
my_glacier = Glacier(bed, mb)

If we simply call the glacier we get some basic statistics about it

In [ ]:
# Prints what is currently known about the glacier.
my_glacier

Since we haven't actually grown the glacier yet, the glacier has no volume, length.
If you didn't pass an mass balance during initialisation it also doesn't have an ELA yet.
This can be added later.

The glacier has a method for plotting:

In [ ]:
# A simple plotting method of the current glacier state.
my_glacier.plot()

In [ ]:
# This will raise an error
my_glacier.plot_history()

With a mass balance, we can let the glacier progress (grow/shrink).
There are two methods for this. `Glacier.progress_to_year(year=)`  progresses the glacier until the specified year.
If it grows or shrinks depends on the mass balance.
The second method is the `progress_to_equilibrium()` which progress the glacier until an equilibrium state is reached.

In [ ]:
my_glacier.progress_to_year(60)

In [ ]:
my_glacier.plot()

Alternatively we can also create the glacier without the mass balance object, by specifying the ELA and mass balance gradient. 
This can be done in any order.

In [ ]:
# New glacier
my_glacier2 = Glacier(bed)
# Set the ela to 3000 m.
my_glacier2.ELA = 3400

In [ ]:
# This will not return anything
my_glacier2.mass_balance

In [ ]:
# Set the mass balance gradient to 7 mm/m
my_glacier2.mb_gradient = 7

In [ ]:
# Now this will return the mass balance description.
my_glacier2.mass_balance

In [ ]:
# Progress the glacier until year 60
my_glacier2.progress_to_year(60)

In [ ]:
my_glacier2.age

In [ ]:
# Outputs have now been updated.
my_glacier2

In [ ]:
# Plotting the glacier now has more information. Glacier surface and ELA.
my_glacier2.plot()

#### Copy a glacier
We can also initialise a new glacier based on an already existing glacier. Simply provide a glacier under the `copy` keyword to the glacier class:

In [ ]:
glacier2 = my_glacier.copy()

This glacier can now be progressed/changed independently from the original glacier.

In [ ]:
# Change the sliding parameter
glacier2.basal_sliding = 5.7e-20
glacier2.progress_to_year(150)
glacier2.plot()

In [ ]:
# Progress the original glacier to year 150
my_glacier.progress_to_year(150)
my_glacier.plot()

## Take a look at the history of the glacier
For this the glacier has the history attribute.
This is a dataset that contains some useful diagnostics about the glacier.

In [ ]:
my_glacier.history

We can plot the history of some attributes of the glacier quickly

In [ ]:
my_glacier.plot_history()

Progress it for a little longer

In [ ]:
my_glacier.progress_to_year(210)
my_glacier.plot()

In [ ]:
my_glacier.plot_history()

Let's progress the glacier to equilibrium.

In [ ]:
# Grow the glacier to equilbrium
my_glacier.progress_to_equilibrium()

In [ ]:
# Plot the glacier again.
my_glacier.plot()

### State history
We can also access the previous states of the glacier under the `.state_history` attribute.
This is an dataset, similar to the history attribute, which provide us with information about the geometry of the glacier, such as the ice thickness.

In [ ]:
my_glacier.state_history

This contains a lot of information, it is nicer to visualise some of the data with the corresponding plotting function `.plot_state_history()'.

In [ ]:
# Default this plot states at 50 year intervals, but can be specified
# with the interval argument.
my_glacier.plot_state_history()

If we change any of the mass balance parameters, it will internally update the mass balance of the glacier. Growing the glacier will start in the previous state but with the new mass balance. Hence, the glacier will either shrink or grow depending on what is changed. As of now, the glacier is not saving the previous states so one can not compare the glacier before and after the change. One could imagine that glacier to have a mechanism that saves states if the mass balance is changed or something similar. But it also nice to keep the glacier class as light as possible, which is why we have the glacier collection. 

Other methods/tweaks that could be interesting for the glacier class:
- ~~Grow and save in order to plot the evolution~~
- ~~Surging, however this might be it's own subclass.~~ See further down.
- ~~Response time~~

There is a method for plotting the mass balance.

In [ ]:
my_glacier.plot_mass_balance()

## Climate change scenario
We can set up a climate change scenario for our glacier. This is done through the `add_temperature_bias(bias, duration)` method.
This will set up a scenario for the glacier so the bias ($\pm$) is reached after the specified duration (years).
The temperature at the glacier is then increased in linear steps annually during the progression.
First we create a glacier and run it to equilibrium.
Then we add a temperature bias scenario and run to equilibrium again.

In [ ]:
# Create a complex bed.
bed = GlacierBed(altitudes=(3800, 3300, 3000, 100),
                 widths=(1200, 900, 800, 600), slope=0.2)
# Create a simple bed
# bed = GlaGlacierBed(top=3800, bottom=2100, width=300)

In [ ]:
# Create a mass balance.
mb = MassBalance(3000, 7)
# Create the glacier.
my_glacier = Glacier(bed, mb)

In [ ]:
 my_glacier.progress_to_equilibrium()

In [ ]:
my_glacier.age

In [ ]:
my_glacier.plot()

In [ ]:
my_glacier.plot_history()

We then add a temperature bias of $+1.5C$, which will take 100 years.

In [ ]:
# Add a temperature bias to the glacier which will take 100 years.
my_glacier.add_temperature_bias(1.5, 100)
# Progress to equilbirum again. This will first complete the temperature 
#scenario and then continue to the equilibrium state.
my_glacier.progress_to_equilibrium()

In [ ]:
my_glacier.plot_history()

By comparing the plots above we can see that in addition to our 100 year temperature evolution the glacier took an additional 28 years to reach equilibrium.

## Response time
The response time of a glacier is available under the attribute `.response_time`.
However, this is only defined when the glacier has two or more equilibrium states in its history.
Hence, this requires the user to setup a glacier and perform experiments so that this is fulfilled.

This is achieved by setting up a temperature change scenario.

In [ ]:
# We start with a new glacier with a simple bed
bed = GlacierBed(top=3400, bottom=1500, width=300)
# Mass balance
mass_balance = MassBalance(ELA=3000, gradient=5)
# Initiate the glacier
glacier = Glacier(bed, mass_balance)
# Progress to equilibrium
glacier.progress_to_equilibrium()
glacier

In [ ]:
# View the equilibrium states
glacier.eq_states

Since the glacier only has one equilibrium state in its history, the response time is not defined

In [ ]:
# Will raise an error.
glacier.response_time

We set up a temperature change scenario and progress to a second equilibrium state

In [ ]:
# Add an abrpupt temperature bias scenario
# At the moment the calculation of the response time does not take
# the time progression of the temperature scenario into account,
# hence we set it to 1 year
glacier.add_temperature_bias(1., 1)
glacier.progress_to_equilibrium()

In [ ]:
# Take a look at the eq. states again.
glacier.eq_states

The response time is now defined.

In [ ]:
# Response time in years
glacier.response_time

It will be updated every time we add a new equilibrium state, calculated based on the last two eq. states.
In theory the response time should stay constant for the glacier, no matter the temperature scenario (raised/lowered temperature)

# The the glacier collection
The `GlacierCollection()` class is made to make it simple to work with multiple glaciers. As of now, we have to define the glaciers separately and put them in the collection. But it is not hard to imagine that it could also have a method to populate it with a number of glaciers.

In [ ]:
# This initiates an empty collection.
collection = GlacierCollection()

In [ ]:
collection

In [ ]:
# As of now, it is empty.
collection.glaciers

Let's create another two glaciers to add to the collection

In [ ]:
# lets create two other glaciers.
# bed = GlacierBed(top=3400, bottom=1400, width=300)
bed = GlacierBed(altitudes=[3400, 2800, 2000, 1500],
                 widths=[400, 600, 600, 500])
# They have the same bed.
glacier1 = Glacier(bed)
glacier1.ELA = 3000
# But different mb gradients
glacier1.mb_gradient = 7
# lets create two other glaciers.
# Here we copy the first one, but one could just define new ones a done
# above.
glacier2 = glacier1.copy()
glacier3 = glacier1.copy()

In [ ]:
collection

We add glaciers to the collection with the `.add()` method:

In [ ]:
# Add the first glacier
collection.add([glacier1, glacier2, glacier3])
# Change some ice flow stuff
# No problem to change the glaciers within the collection
# "outside" of the collection.
glacier2.creep = glacier2.creep * 10
glacier3.creep = glacier3.creep / 10

In [ ]:
# Now this has some content, with a pretty representation, this is however not a full dataframe.
collection

In [ ]:
# This is one of the glaciers in the collection.
# We can modify it with methods of the Glacier object.
collection.glaciers[1]

In [ ]:
# One can also add the glaciers to the collection separately.
# collection.add(glacier1)
# Note that this will fail since glacier2 is already in the
# collection.
collection.add(glacier2)

As a glacier, the glacierCollection also has a `.plot()` method.

In [ ]:
collection.plot()

But our glaciers haven't grown anything yet so this will be empty. The collection has the same methods for growing as the individual glaciers

In [ ]:
# Grow the glacier in the collection to year ...
collection.progress_to_year(100)

In [ ]:
# If we plot it again
collection.plot()

And we can grow the glaciers until equilibrium

In [ ]:
collection.progress_to_equilibrium()

In [ ]:
collection.plot()

In [ ]:
collection.plot_history()

In [ ]:
# Add a temperature bias to one of the glaciers in the collection.
glacier2.add_temperature_bias(1., 50)

In [ ]:
#  Glaciers in collection will now have different ages.
collection.progress_to_equilibrium()

In [ ]:
collection.plot_history()

## Another way of populating the collection
It is also possible to populate the collection with the `.fill()` method

In [ ]:
new_collection = GlacierCollection()
# lets create a glacier.
# bed = GlacierBed(top=3400, bottom=1400, width=300)
bed = GlacierBed(altitudes=[3400, 2800, 2000, 1500],
                 widths=[400, 600, 600, 500])
# They have the same bed.
glacier = Glacier(bed)
glacier.ELA = 3000
# But different mb gradients
glacier.mb_gradient = 7

In [ ]:
new_collection

In [ ]:
# Fill it
# We provide an initial glacier, the number of glaciers we want in the
# collection, and optionally a dictionary with key-value pairs for attributes
# to change. More on this later.
new_collection.fill(glacier=glacier, n=5,
                    attributes_to_change={'ELA': [3000, 2800, 3000, 2600, 2500]}
                   )

In [ ]:
new_collection

There is also a method for batch changing attributes of glaciers in the collection.

In [ ]:
# This will change the ELA and mb_gradient of the glaciers in the collection.
new_collection.change_attributes(attributes_to_change=
                             {'mb_gradient': [5, 4, 7, 8, 15]}
                            )
new_collection

In [ ]:
new_collection.progress_to_year(55)

In [ ]:
new_collection.plot()

In [ ]:
new_collection.plot_history()

Other attributes one can change are listed in the description of the method:

In [ ]:
?collection.change_attributes

More things we can add to the `GlacierCollection`, e.g.
- ~~Populate the collection with n copies of one initial glacier. Then change the attributes.~~
- Plot mass balances?
- ???

## Surging Glacier
 The surging glacier is provided by another class. It behaves mostly like the glacier, but with some tweaks

In [ ]:
# Create a complex bed.
bed = GlacierBed(altitudes=(3800, 3300, 3000, 2100),
                 widths=(1200, 800, 800, 600), slope=0.2)
# We're using the bed from earlier.
surging_glacier = SurgingGlacier(bed)

In [ ]:
surging_glacier

In [ ]:
# Same as the Glacier 
surging_glacier.ELA = 3600
surging_glacier.mb_gradient = 7
surging_glacier.basal_sliding = 5.7e-20
# Some of the new attributes of the surging glacier
surging_glacier.normal_years = 100
surging_glacier.surging_years = 10
surging_glacier.basal_sliding_surge = 5.7e-20 * 10

In [ ]:
surging_glacier.progress_to_year(200)
surging_glacier.plot()

In [ ]:
# Plot the history
surging_glacier.plot_history()

In [ ]:
# We can easlity to continue the progress of the glacier, this does not
# re-run the glacier, but continues from the current state.
surging_glacier.progress_to_year(500)
surging_glacier.plot_history()

In [ ]:
# Surging glaciers do not have the progress_to_equilibrium method.
surging_glacier.progress_to_equilibrium()